In [1]:
%store -r word_df
%store -r num_df
%store -r single_df
%store -r word_vector

import pandas as pd
import numpy as np

y = word_df['target']
X = num_df.join(single_df)
X = X.drop(columns=['target'])
print(X.head())
print(y.head())

   search_word_Siemens  search_word_cats  search_word_christmas  \
0                    1                 0                      0   
1                    1                 0                      0   
2                    1                 0                      0   
3                    1                 0                      0   
4                    1                 0                      0   

   search_word_donald trump  search_word_dude perfect  search_word_gaming  \
0                         0                         0                   0   
1                         0                         0                   0   
2                         0                         0                   0   
3                         0                         0                   0   
4                         0                         0                   0   

   search_word_golf  search_word_horror  search_word_iphone  \
0                 0                   0                   0   
1       

In [2]:
X.columns


Index(['search_word_Siemens', 'search_word_cats', 'search_word_christmas',
       'search_word_donald trump', 'search_word_dude perfect',
       'search_word_gaming', 'search_word_golf', 'search_word_horror',
       'search_word_iphone', 'search_word_laugh', 'search_word_mac',
       'search_word_mars', 'search_word_minecraft', 'search_word_movies',
       'search_word_music', 'search_word_python', 'search_word_speedrun',
       'search_word_sports', 'search_word_vacation',
       'search_word_video blogging', 'contentDetails.definition_hd',
       'contentDetails.definition_sd', 'commentCount', 'dislikeCount',
       'likeCount', 'view_bucket', 'age_int', 'title_char_count',
       'title_word_count', 'title_punctuation_count',
       'title_upper_case_word_count', 'title_stopwords_count',
       'desc_char_count', 'desc_word_count', 'desc_word_density',
       'desc_punctuation_count', 'desc_title_word_count',
       'desc_upper_case_word_count', 'desc_stopwords_count', 'tags_char_co

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)



In [4]:
from sklearn.preprocessing import FunctionTransformer


# Obtain the text data: get_text_data
get_text_data = FunctionTransformer(lambda x: x['words'], validate=False)

# Obtain the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[['search_word_Siemens', 'search_word_cats', 'search_word_christmas',
       'search_word_donald trump', 'search_word_dude perfect',
       'search_word_gaming', 'search_word_golf', 'search_word_horror',
       'search_word_iphone', 'search_word_laugh', 'search_word_mac',
       'search_word_mars', 'search_word_minecraft', 'search_word_movies',
       'search_word_music', 'search_word_python', 'search_word_speedrun',
       'search_word_sports', 'search_word_vacation',
       'search_word_video blogging', 'contentDetails.definition_hd',
       'contentDetails.definition_sd', 'commentCount', 'dislikeCount',
       'likeCount', 'view_bucket', 'age_int', 'title_char_count',
       'title_word_count', 'title_punctuation_count',
       'title_upper_case_word_count', 'title_stopwords_count',
       'desc_char_count', 'desc_word_count', 'desc_word_density',
       'desc_punctuation_count', 'desc_title_word_count',
       'desc_upper_case_word_count', 'desc_stopwords_count', 'tags_char_count',
       'tags_word_count', 'tags_word_density', 'tags_punctuation_count',
       'tags_upper_case_word_count', 'tags_stopwords_count']], validate=False)

# Fit and transform the text data: just_text_data
just_text_data = get_text_data.fit_transform(X_train)

# Fit and transform the numeric data: just_numeric_data
just_numeric_data = get_numeric_data.fit_transform(X_train)

# Print head to check results
print('Text Data')
print(just_text_data.head())
print('\nNumeric Data')
print(just_numeric_data.head())

Text Data
464    donald trump President Donald Trump Then And N...
97     cats 10 Things CATS HATE That You Should Avoid...
936    movies HOT Kung-Fu Martial Arts Chinese 2018 ƒ...
204    minecraft Minecraft Fnaf: Lolbit Gets A Makeov...
306    gaming Wengie v. Marshmello 1v1 Street Fighter...
Name: words, dtype: object

Numeric Data
     search_word_Siemens  search_word_cats  search_word_christmas  \
464                    0                 0                      0   
97                     0                 1                      0   
936                    0                 0                      0   
204                    0                 0                      0   
306                    0                 0                      0   

     search_word_donald trump  search_word_dude perfect  search_word_gaming  \
464                         1                         0                   0   
97                          0                         0                   0   
936         

In [5]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score

process_and_join_features = FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )

# Instantiate nested pipeline: pl
pl = Pipeline([
        ('union', process_and_join_features),
        ('clf', OneVsRestClassifier(LogisticRegression(random_state=42)))
    ])

pl.fit(X_train, y_train)
# Fit pl to the training data
y_pred = pl.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.46774193548387094
[[59 23  3  1  0  0]
 [14 17  7  0  1  0]
 [10 17 16 11  2  0]
 [ 9  1  7  4  3  2]
 [ 9  0  0  1 10  3]
 [ 6  0  0  0  2 10]]
              precision    recall  f1-score   support

           0       0.55      0.69      0.61        86
           1       0.29      0.44      0.35        39
           2       0.48      0.29      0.36        56
           3       0.24      0.15      0.19        26
           4       0.56      0.43      0.49        23
           5       0.67      0.56      0.61        18

   micro avg       0.47      0.47      0.47       248
   macro avg       0.46      0.43      0.43       248
weighted avg       0.47      0.47      0.46       248

0.4570607171001012


In [6]:
# Randp,
from sklearn.ensemble import RandomForestClassifier

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', RandomForestClassifier(random_state=42))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

y_pred = pl.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.43548387096774194
[[79  2  2  1  2  0]
 [32  4  2  0  1  0]
 [33  4 14  3  2  0]
 [12  2  5  5  2  0]
 [10  1  7  2  2  1]
 [10  0  1  1  2  4]]
              precision    recall  f1-score   support

           0       0.45      0.92      0.60        86
           1       0.31      0.10      0.15        39
           2       0.45      0.25      0.32        56
           3       0.42      0.19      0.26        26
           4       0.18      0.09      0.12        23
           5       0.80      0.22      0.35        18

   micro avg       0.44      0.44      0.44       248
   macro avg       0.43      0.30      0.30       248
weighted avg       0.42      0.44      0.37       248

0.3697356496246408


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [7]:
from sklearn.linear_model import SGDClassifier
    
pl = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('imputer', Imputer())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
            ]))
            ]
    )),
      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

y_pred = pl.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.2620967741935484
[[53 21  8  0  4  0]
 [21  3 14  0  1  0]
 [47  0  9  0  0  0]
 [25  0  1  0  0  0]
 [22  0  1  0  0  0]
 [18  0  0  0  0  0]]
              precision    recall  f1-score   support

           0       0.28      0.62      0.39        86
           1       0.12      0.08      0.10        39
           2       0.27      0.16      0.20        56
           3       0.00      0.00      0.00        26
           4       0.00      0.00      0.00        23
           5       0.00      0.00      0.00        18

   micro avg       0.26      0.26      0.26       248
   macro avg       0.11      0.14      0.11       248
weighted avg       0.18      0.26      0.20       248

0.1957856221502737


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
from sklearn.neighbors import KNeighborsClassifier 

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('knn', KNeighborsClassifier(n_neighbors=11) )
    ])

# Fit to the training data
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.5362903225806451
[[60  7  5  4  9  1]
 [12 23  4  0  0  0]
 [ 7 18 25  5  1  0]
 [ 2  2  8  9  5  0]
 [ 2  0  2  6 13  0]
 [10  0  0  1  4  3]]
              precision    recall  f1-score   support

           0       0.65      0.70      0.67        86
           1       0.46      0.59      0.52        39
           2       0.57      0.45      0.50        56
           3       0.36      0.35      0.35        26
           4       0.41      0.57      0.47        23
           5       0.75      0.17      0.27        18

   micro avg       0.54      0.54      0.54       248
   macro avg       0.53      0.47      0.46       248
weighted avg       0.55      0.54      0.53       248

0.5272952557490169


In [9]:
from xgboost import XGBClassifier

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('xgb', XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10, seed=42)) 
    ])

# Fit to the training data
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

                     

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.8145161290322581
[[77  9  0  0  0  0]
 [ 7 31  1  0  0  0]
 [ 1 12 40  3  0  0]
 [ 0  0  3 22  1  0]
 [ 0  0  0  0 23  0]
 [ 3  0  0  0  6  9]]
              precision    recall  f1-score   support

           0       0.88      0.90      0.89        86
           1       0.60      0.79      0.68        39
           2       0.91      0.71      0.80        56
           3       0.88      0.85      0.86        26
           4       0.77      1.00      0.87        23
           5       1.00      0.50      0.67        18

   micro avg       0.81      0.81      0.81       248
   macro avg       0.84      0.79      0.79       248
weighted avg       0.84      0.81      0.81       248

0.8140322895166421


In [10]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_GBC = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('gbc', GradientBoostingClassifier(random_state=42)) 
    ])

# Fit to the training data
pipe_GBC.fit(X_train, y_train)
y_pred = pipe_GBC.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.842741935483871
[[76  9  1  0  0  0]
 [ 9 27  3  0  0  0]
 [ 0 12 44  0  0  0]
 [ 0  0  4 22  0  0]
 [ 0  0  0  0 23  0]
 [ 1  0  0  0  0 17]]
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        86
           1       0.56      0.69      0.62        39
           2       0.85      0.79      0.81        56
           3       1.00      0.85      0.92        26
           4       1.00      1.00      1.00        23
           5       1.00      0.94      0.97        18

   micro avg       0.84      0.84      0.84       248
   macro avg       0.88      0.86      0.87       248
weighted avg       0.86      0.84      0.85       248

0.8474015072596831


In [11]:

n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
min_samples_leaf = [1, 2, 4, 6, 8] 
min_samples_split = [2, 4, 6, 10]
max_features = ['auto', 'sqrt', 'log2', None]


random_grid = {
    'gbc__n_estimators': n_estimators,
    'gbc__max_depth': max_depth,
    'gbc__min_samples_leaf': min_samples_leaf,
    'gbc__min_samples_split': min_samples_split,
    'gbc__max_features': max_features}


In [12]:
#randomized search with GBC
from sklearn.model_selection import RandomizedSearchCV

gbc_random = RandomizedSearchCV(estimator = pipe_GBC, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

gbc_random.fit(X_train, y_train)
y_pred = gbc_random.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.war

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

...

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

0.8306451612903226
[[77  8  1  0  0  0]
 [ 9 27  3  0  0  0]
 [ 0 14 40  2  0  0]
 [ 0  0  4 22  0  0]
 [ 0  0  0  0 23  0]
 [ 1  0  0  0  0 17]]
              precision    recall  f1-score   support

           0       0.89      0.90      0.89        86
           1       0.55      0.69      0.61        39
           2       0.83      0.71      0.77        56
           3       0.92      0.85      0.88        26
           4       1.00      1.00      1.00        23
           5       1.00      0.94      0.97        18

   micro avg       0.83      0.83      0.83       248
   macro avg       0.86      0.85      0.85       248
weighted avg       0.84      0.83      0.83       248

0.8343926162873113


In [13]:
# Pipeline with feature importance ability 
from sklearn import feature_selection

pipe_XGB = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('tfidf', TfidfVectorizer())
                ]))
             ]
        )),
        ('xgb', XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
 colsample_bytree=0.8, objective= 'multi:softmax', nthread=4, scale_pos_weight=1,seed=42))
    ])

pipe_XGB.fit(X_train, y_train)
y_pred = pipe_XGB.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.8225806451612904
[[77  9  0  0  0  0]
 [12 25  2  0  0  0]
 [ 0 10 39  7  0  0]
 [ 0  0  3 23  0  0]
 [ 0  0  0  0 23  0]
 [ 1  0  0  0  0 17]]
              precision    recall  f1-score   support

           0       0.86      0.90      0.88        86
           1       0.57      0.64      0.60        39
           2       0.89      0.70      0.78        56
           3       0.77      0.88      0.82        26
           4       1.00      1.00      1.00        23
           5       1.00      0.94      0.97        18

   micro avg       0.82      0.82      0.82       248
   macro avg       0.85      0.84      0.84       248
weighted avg       0.83      0.82      0.82       248

0.8236565848648049


In [14]:
random_XGB = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('xgb', XGBClassifier(learning_rate =0.1, n_estimators=1000, gamma=0, subsample=0.8,
 colsample_bytree=0.8, objective= 'multi:softmax', nthread=4, scale_pos_weight=1,seed=42))
        ])

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [15]:
from sklearn.model_selection import GridSearchCV

param_test1 = {
 'xgb__max_depth':range(3,10,2),
 'xgb__min_child_weight':range(1,6,2)
}

In [16]:
#pass in full pipeline into GridSearchCV( )
gsearch1 = GridSearchCV(random_XGB, param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=5)

#Check objective function for XGBClassifier documentation.  
gsearch1.fit(X_train,y_train)


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000000000AAEE2F0>, inv_kw_args=None,
          inverse_func=None...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=0.8))]),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'xgb__max_depth': range(3, 10, 2), 'xgb__min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [17]:
print("Tuned Grid Search Parameters: {}".format(gsearch1.best_params_)) 

y_pred = gsearch1.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

Tuned Grid Search Parameters: {'xgb__max_depth': 9, 'xgb__min_child_weight': 1}
0.8306451612903226
[[75 11  0  0  0  0]
 [10 26  3  0  0  0]
 [ 0  9 43  4  0  0]
 [ 0  0  4 22  0  0]
 [ 0  0  0  0 23  0]
 [ 1  0  0  0  0 17]]
              precision    recall  f1-score   support

           0       0.87      0.87      0.87        86
           1       0.57      0.67      0.61        39
           2       0.86      0.77      0.81        56
           3       0.85      0.85      0.85        26
           4       1.00      1.00      1.00        23
           5       1.00      0.94      0.97        18

   micro avg       0.83      0.83      0.83       248
   macro avg       0.86      0.85      0.85       248
weighted avg       0.84      0.83      0.83       248

0.8337842745132137


In [18]:
grid_GB = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('gbc', GradientBoostingClassifier(random_state=42)) 
        ])

gsearch = RandomizedSearchCV(grid_GB, random_grid, scoring='accuracy',n_jobs=4,iid=False, cv=5, random_state=42)

#Check objective function for XGBClassifier documentation.  
gsearch.fit(X_train,y_train)
y_pred = gsearch.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Con

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.8306451612903226
[[76  9  1  0  0  0]
 [ 9 28  2  0  0  0]
 [ 0 14 40  2  0  0]
 [ 0  0  4 22  0  0]
 [ 0  0  0  0 23  0]
 [ 1  0  0  0  0 17]]
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        86
           1       0.55      0.72      0.62        39
           2       0.85      0.71      0.78        56
           3       0.92      0.85      0.88        26
           4       1.00      1.00      1.00        23
           5       1.00      0.94      0.97        18

   micro avg       0.83      0.83      0.83       248
   macro avg       0.87      0.85      0.86       248
weighted avg       0.85      0.83      0.84       248

0.8351916394493909


In [19]:
# Try Random Forest initially without parameter tuning 
from sklearn.ensemble import RandomForestClassifier

rf_pipe = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('rf', RandomForestClassifier(random_state=42)) 
        ])

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'rf__n_estimators': n_estimators,
               'rf__max_features': max_features,
               'rf__max_depth': max_depth,
               'rf__min_samples_split': min_samples_split,
               'rf__min_samples_leaf': min_samples_leaf,
               'rf__bootstrap': bootstrap}

In [21]:
rsearch = RandomizedSearchCV(rf_pipe, random_grid, scoring='accuracy',n_jobs=4,iid=False, cv=5, random_state=42)

In [22]:

rsearch.fit(X_train,y_train)
y_pred = rsearch.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_true=y_test, y_pred=y_pred, average='weighted'))

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0.4637096774193548
[[77  7  0  1  1  0]
 [27  9  2  1  0  0]
 [26  9 15  3  3  0]
 [ 9  4  7  4  2  0]
 [11  0  3  2  6  1]
 [10  0  0  1  3  4]]
              precision    recall  f1-score   support

           0       0.48      0.90      0.63        86
           1       0.31      0.23      0.26        39
           2       0.56      0.27      0.36        56
           3       0.33      0.15      0.21        26
           4       0.40      0.26      0.32        23
           5       0.80      0.22      0.35        18

   micro avg       0.46      0.46      0.46       248
   macro avg       0.48      0.34      0.35       248
weighted avg       0.47      0.46      0.42       248

0.4169338843211722
